In [2]:
import sys
sys.path.append('..\\helpers')

import numpy as np
import skimage.segmentation as seg
import skimage.color as color
import skimage.io as io
import cv2
import os
import math
from morphological_sifter import MorphologicalSifter
from display import plot_figures, read_img, convert_16_to_8_bit
import matplotlib.pyplot as plt

mms = MorphologicalSifter()

# To allow auto reload to this notebook after modifying any external file imported
%load_ext autoreload
%autoreload 2

In [3]:
try:

    input_img = cv2.imread("..\\..\\project\\dataset\\output\\mms\\20587928_8dbbd4e51f549ff0_MG_R_ML_ANON_3.tif")
    
    if input_img is None:
        raise Exception("Cannot open image")

    # convert to float & reshape to a [3 x W*H] Mat 
    # (so every pixel is on a row of its own)
    data = input_img.astype(np.float32)
    data = data.reshape(1, -1, 3)

    # do kmeans
    _, labels, centers = cv2.kmeans(
        data, 2, None, 
        criteria=(cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 1.0), 
        attempts=3, 
        flags=cv2.KMEANS_PP_CENTERS
    )

    # replace pixel values with their center value:
    p = data[0]
    for i in range(data.shape[1]):
        center_id = labels[i]
        p[i] = centers[center_id]

    # back to 2D, and uchar:
    img = data.reshape(input_img.shape)
    img = np.clip(img, 0, 255).astype(np.uint8)

    cv2.imshow("K-means clustering result", img)

    # smooth boundaries
    img = cv2.medianBlur(img, 7)

    # select largest foreground component
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
    objects, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    max_area_idx = -1
    max_area = -1
    for i in range(len(objects)):
        A = cv2.contourArea(objects[i])
        if A > max_area:
            max_area = A
            max_area_idx = i

    # display final result
    cv2.drawContours(input_img, objects, -1, (0, 0, 0), 1, cv2.LINE_AA)
    cv2.drawContours(input_img, objects, max_area_idx, (22, 14, 177), 2, cv2.LINE_AA)

    cv2.imshow("Segmentation result", input_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

except Exception as e:
    print(str(e))


In [51]:
try:

    input_img = cv2.imread("..\\..\\project\\dataset\\output\\mms\\20587612_f4b2d377f43ba0bd_MG_R_CC_ANON_1.tif")
    
    if input_img is None:
        raise Exception("Cannot open image")

    # convert to float & reshape to a [3 x W*H] Mat 
    # (so every pixel is on a row of its own)
    data = input_img.astype(np.float32)
    data = data.reshape(1, -1, 3)

    # do kmeans
    _, labels, centers = cv2.kmeans(
        data, 2, None, 
        criteria=(cv2.TERM_CRITERIA_MAX_ITER + cv2.TERM_CRITERIA_EPS, 10, 1.0), 
        attempts=3, 
        flags=cv2.KMEANS_PP_CENTERS
    )

    # replace pixel values with their center value:
    p = data[0]
    for i in range(data.shape[1]):
        center_id = labels[i]
        p[i] = centers[center_id]

    # back to 2D, and uchar:
    img = data.reshape(input_img.shape)
    img = np.clip(img, 0, 255).astype(np.uint8)

    # smooth boundaries
    img = cv2.medianBlur(img, 7)

    # select largest foreground component
    img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    _, img = cv2.threshold(img, 0, 255,  cv2.THRESH_OTSU)
    
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    # Create a blank image to draw contours on
    result = np.zeros_like(img)

    # Loop through contours and eliminate those outside the desired area range
    for cnt in contours:
        area = cv2.contourArea(cnt)
        if 150 < area < 2450:
            cv2.drawContours(result, [cnt], 0, (255, 255, 255), -1)
    
    # display final result
    # cv2.drawContours(input_img, contours, -1, (22, 14, 177), 2, cv2.LINE_AA)

    cv2.imshow("input_img result", input_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    
    cv2.imshow("Segmentation result", result)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

except Exception as e:
    print(str(e))


In [10]:
import cv2
import numpy as np


input_img = cv2.imread("..\\..\\project\\dataset\\output\\mms\\20587664_f4b2d377f43ba0bd_MG_R_ML_ANON_1.tif")

# Convert to float and reshape to [3 x W*H] matrix
data = input_img.astype(np.float32)
data = data.reshape(-1, 3)

# Perform k-means clustering
criteria = (cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_PP_CENTERS
compactness, labels, centers = cv2.kmeans(data, 2, None, criteria, 3, flags)

# Replace pixel values with their center value
for i in range(data.shape[0]):
    center_id = labels[i][0]
    data[i] = centers[center_id]

# Reshape back to 2D and convert to uchar
img = data.reshape(input_img.shape).astype(np.uint8)

# Display the k-means clustering result
cv2.imshow("K-means clustering result", img)
cv2.waitKey(0)

# Smooth boundaries
img = cv2.medianBlur(img, 7)

# Select the largest foreground component
img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
_, img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
max_area_idx = -1
max_area = -1
for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    if area > max_area:
        max_area = area
        max_area_idx = i

# Display the segmentation result
cv2.drawContours(input_img, contours, -1, (111, 22, 222), 2, cv2.LINE_AA)
cv2.drawContours(input_img, contours, max_area_idx, (22, 14, 177), 2, cv2.LINE_AA)
cv2.imshow("Segmentation result", input_img)
cv2.waitKey(0)


-1

In [52]:
import cv2
import numpy as np

# Load image
input_img = cv2.imread("..\\..\\project\\dataset\\output\\mms\\20587928_8dbbd4e51f549ff0_MG_R_ML_ANON_3.tif")
if input_img is None:
    raise Exception("Cannot open image")

# Convert to float and reshape to [3 x W*H] matrix
data = input_img.astype(np.float32)
data = data.reshape(-1, 3)

# Perform k-means clustering
criteria = (cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_PP_CENTERS
compactness, labels, centers = cv2.kmeans(data, 2, None, criteria, 3, flags)

# Replace pixel values with their center value
for i in range(data.shape[0]):
    center_id = labels[i][0]
    data[i] = centers[center_id]

# Reshape back to 2D and convert to uchar
img = data.reshape(input_img.shape).astype(np.uint8)

# Display the k-means clustering result
cv2.imshow("K-means clustering result", img)
cv2.waitKey(0)

# Smooth boundaries
img = cv2.medianBlur(img, 7)

# Convert to grayscale and perform thresholding
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
# _, img_thresh = cv2.threshold(img_gray, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)
img_thresh = cv2.adaptiveThreshold(img_gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV, 11, 4)

# Find contours in the binary image
contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

result = np.zeros_like(img_thresh)

for i, contour in enumerate(contours):
    area = cv2.contourArea(contour)
    print(area)
    if 1000 < area < 90000:
        cv2.drawContours(result, [contour], 0, (255, 255, 255), -1)

# Draw contours on the original image
cv2.drawContours(input_img, contours, -1, (22, 14, 177), 2, cv2.LINE_AA)
cv2.imshow("Segmentation result", input_img)
cv2.waitKey(0)


1215.0
76297.0
1660.0
2597.5
1022.0


-1

In [12]:
import cv2
import numpy as np

input_img = cv2.imread("..\\..\\project\\dataset\\output\\mms\\20587664_f4b2d377f43ba0bd_MG_R_ML_ANON_1.tif")

# Convert to float and reshape to [3 x W*H] matrix
data = input_img.astype(np.float32)
data = data.reshape(-1, 3)

# Perform k-means clustering
criteria = (cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
flags = cv2.KMEANS_PP_CENTERS
compactness, labels, centers = cv2.kmeans(data, 2, None, criteria, 3, flags)

# Replace pixel values with their center value
for i in range(data.shape[0]):
    center_id = labels[i][0]
    data[i] = centers[center_id]

# Reshape back to 2D and convert to uchar
img = data.reshape(input_img.shape).astype(np.uint8)

# Display the k-means clustering result
cv2.imshow("K-means clustering result", img)
cv2.waitKey(0)

# Smooth boundaries
img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
img_blur = cv2.medianBlur(img_gray, 7)

# Threshold the image to create a binary mask
_, img_thresh = cv2.threshold(img_blur, 0, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

# Find contours in the binary mask
contours, _ = cv2.findContours(img_thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Select regions with circular shape
selected_contours = []
for contour in contours:
    perimeter = cv2.arcLength(contour, True)
    approx = cv2.approxPolyDP(contour, 0.04 * perimeter, True)
    if len(approx) >= 5:
        area = cv2.contourArea(contour)
        circularity = (4 * np.pi * area) / (perimeter * perimeter)
        print(area)
        print(circularity)
        if circularity > 0.7:
            selected_contours.append(contour)

# Draw the selected circular contours on the original image
result_img = input_img.copy()
cv2.drawContours(result_img, selected_contours, -1, (22, 14, 177), 2, cv2.LINE_AA)

# Display the result
cv2.imshow("Circular Regions", result_img)
cv2.waitKey(0)


72.5
0.7705759879205338


-1